In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1337)

In [ ]:
# Get the shakespeare karpathy dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt', 'r') as f:
    text = f.read()

In [ ]:
print("Num of chars:", len(text))

In [ ]:
text[:1000]

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars) # our vocab size is all the unique characters used in the Shakespeare data
print(''.join(chars))
print(vocab_size)

In [ ]:
# Now we want to tokenize our characters by mapping each character to a number
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder takes a string, outputs a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder takes a list of integers, outputs a string

In [ ]:
print(encode("hello"))
print(decode(encode("hello")))

In [ ]:
# Now encode the entire text and save it to a tensor
data = torch.tensor(encode(text), dtype=torch.long)
data.shape, data.dtype

In [ ]:
#split the data into train and val sets
n = int(0.9*len(data))
train = data[:n]
val = data[n:]
print(len(train), len(val))

In [ ]:
block_size = 8
train[:block_size+1]
# We use a sequence length of 8, but we take the data up to and including sequence length# + 1
# because we want to know for 8 chars, what the pred for next char should be

In [ ]:
x = train[:block_size]
y = train[1:block_size+1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"When input is {context}, the target is: {block_size}")

In [ ]:
# Now we want to generate batches to process in parallel
batch_size = 4 # how many batches do we want to process in parallel
block_size = 8 # maximum sequence length for predictions

def get_batch(data, batch_size, block_size):
  indices = torch.randint(0, len(data)-block_size, (batch_size, )) # Creating random starting indices for however many batches we want
  x = torch.stack([data[i : i + block_size] for i in indices])
  y = torch.stack([data[i + 1 : i + block_size + 1] for i in indices])
  return x, y

In [ ]:
# GPT uses a decoder-only structure from the transformer, which doesn't include cross-attention
# First we are going to start with the input embedding
class InputEmbedding(nn.Module):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embedding(x) * math.sqrt(self.d_model)

# Takes tensor of shape (batch size, seq len) -> (batch size, seq len, embedding dim)

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_seq_len):
    super().__init__()
    encoding_matrix = torch.zeros(max_seq_len, d_model)
    position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1) # column vec of positions for vectorization
    div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * -(math.log(10000.0) / d_model))
    # Each index in positional encoding accounts for TWO dimensions in an embedding vector
    # What I do here is create the denominator term for the even set of indices,
    # then apply it to both even and odd dimensions in the embeding
    encoding_matrix[:, 0::2] = torch.sin(position*div_term) # even indices, start at 0 stepsize 2
    encoding_matrix[:, 1::2] = torch.cos(position*div_term) # odd indices, start at 1 stepsize 2
    self.register_buffer("pos_encoding", encoding_matrix)
  def forward(self, x):
    return x + self.pos_encoding[:x.size(1)] # Return positional encodings for the target sequence length

In [ ]:
class ScaledDotProductAttention(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, q, k, v, mask=None):
    d_k = k.size(-1) # Q and K have shape (batch_size, n heads, seq_len, head dim)
    scores = q@k.transpose(-2, -1) / math.sqrt(d_k) # (batch_size, n heads, q_len, k_len)
    if mask is not None:
      scores = scores.masked_fill(mask == 0, float('-inf'))
    attn = F.softmax(scores, dim=-1) # use last dim because we want to sum across keys to get probs for each query
    attn = attn@v # (batch_size, n_heads, q_len, head dim)
    return attn

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, n_heads):
    super().__init__()
    self.n_heads = n_heads
    self.d_head = d_model // n_heads
    self.W_q = nn.Linear(d_model, d_model) # d model is the embedding dimension
    self.W_k = nn.Linear(d_model, d_model)
    self.W_v = nn.Linear(d_model, d_model)
    self.W_o = nn.Linear(d_model, d_model)

  def forward(self, q, k, v, mask=None):
    q = self.W_q(q) # q @ W.T, so shape of all of these is (batch size, seq len, embedding dim)
    k = self.W_k(k)
    v = self.W_v(v)

    batch_size_q, seq_len_q, d_model_q = q.shape
    batch_size_k, seq_len_k, d_model_k = k.shape
    batch_size_v, seq_len_v, d_model_v = v.shape

    q = q.reshape(batch_size_q, seq_len_q, self.n_heads, self.d_head).transpose(1, 2)
      # The original shape of Q here is (batch size, seq len, embedding dim)
      # We want to split the embedding dim into n_heads x d_head, or dimension of each head for a number of heads
    k = k.reshape(batch_size_k, seq_len_k, self.n_heads, self.d_head).transpose(1, 2)
    v = v.reshape(batch_size_v, seq_len_v, self.n_heads, self.d_head).transpose(1, 2)

    attn = ScaledDotProductAttention()(q, k, v, mask) # Shape (batch size, # of heads, seq_len, # embedding dim per head)
    attn_concat = attn.transpose(1, 2).reshape(batch_size_q, seq_len_q, d_model_q) # return axes to original posns before reshaping
    mha_output = self.W_o(attn_concat)

    return mha_output # shape (batch size, seq len, embedding dimension)

In [ ]:
class FeedForward(nn.Module):
  def __init__(self, d_model, d_ff):
    super().__init__()
    self.linear1 = nn.Linear(d_model, d_ff)
    self.linear2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    z1 = self.linear1(x)
    a1 = F.relu(z1)
    z2 = self.linear2(a1)
    return z2

In [ ]:
class LayerNorm(nn.Module):
  def __init__(self, d_model, eps=1e-6):
    super().__init__()
    self.gamma = nn.Parameter(torch.ones(d_model))
    self.beta = nn.Parameter(torch.zeros(d_model))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True) # keep dim to maintain tensor dimensionality
    std = x.std(-1, keepdim=True)
    return self.gamma * (x - mean) / (std + self.eps) + self.beta

# In batch normalization, each feature is normalized across all samples
# In layer normalization, all features are normalized across each sample

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, n_heads, d_ff, dropout_rate):
    super().__init__()
    self.masked_attention = MultiHeadAttention(d_model, n_heads)
    self.norm1 = LayerNorm(d_model)
    self.ff = FeedForward(d_model, d_ff)
    self.norm2 = LayerNorm(d_model)
    self.dropout = nn.Dropout(dropout_rate)

  def forward(self, x, mask):
    x = self.norm1(x + self.dropout(self.masked_attention(x, x, x, mask)))# causal mask for the decoder masked attention
    x = self.norm2(x + self.dropout(self.ff(x)))
    return x

In [ ]:
class MiniGPT(nn.Module):
  def __init__(self, d_model, d_ff, n_heads, dropout_rate, vocab_size, n_layers, max_seq_len):
    super().__init__()
    self.embedding = InputEmbedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding(d_model, max_seq_len)
    self.decoders = nn.ModuleList([DecoderLayer(d_model, n_heads, d_ff, dropout_rate) for _ in range(n_layers)])

    self.final_layernorm = LayerNorm(d_model)

    self.linear_output = nn.Linear(d_model, vocab_size)
    self.max_seq_len = max_seq_len

  def forward(self, x, targets=None):

    batch_size, seq_len = x.shape

    token_embedding = self.embedding(x)
    final_input = self.pos_encoding(token_embedding) # batch size, seq_len, embedding dim

    causal_mask = torch.tril(torch.ones(seq_len, seq_len)).to(x.device)
    causal_mask = causal_mask.unsqueeze(0).unsqueeze(1)

    for decoder in self.decoders:
      final_input = decoder(final_input, causal_mask)

    final_input = self.final_layernorm(final_input)
    logits = self.linear_output(final_input)

    if targets is not None:
      batch_size, seq_len, vocabulary_size = logits.shape
      logits = logits.reshape(batch_size*seq_len, vocabulary_size)
      targets = targets.reshape(batch_size*seq_len)
      loss = F.cross_entropy(logits, targets)
      return logits, loss
    else:
      return logits

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
      idx = idx[:, -self.max_seq_len:]
      logits = self.forward(idx) # shape batch size, seq len, vocab size
      logits_final_token = logits[:, -1, :] # shape of batch size, vocab size (we selected predictions for the next token position)
      probs = F.softmax(logits_final_token, dim=-1) # now we want probabilities of the next token across the vocab size

      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx, idx_next), dim=1) # shape batch size, seq len + 1

    return idx




In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MiniGPT(
    d_model=128,
    d_ff=512,
    n_heads=4,
    dropout_rate=0.1,
    vocab_size=vocab_size,
    n_layers=4,
    max_seq_len=128
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

# Early stopping parameters
patience = 5
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

for step in range(10000):  # adjust iterations as needed
    # Training step
    x, y = get_batch(train, batch_size=64, block_size=128)
    x, y = x.to(device), y.to(device)

    model.train()
    logits, loss = model(x, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation step
    if step % 1000 == 0:  # Check validation every 100 steps
        model.eval()
        with torch.no_grad():
            val_x, val_y = get_batch(val, batch_size=64, block_size=128)
            val_x, val_y = val_x.to(device), val_y.to(device)
            _, val_loss = model(val_x, val_y)

        print(f"Step {step} | Train Loss: {loss.item():.4f} | Val Loss: {val_loss.item():.4f}")

        # Early stopping check
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            patience_counter = 0
            # Save the best model
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at step {step}")
                break

    elif step % 1000 == 0:
        print(f"Step {step} | Train Loss: {loss.item():.4f}")

In [ ]:
prompt = "To be or not to be, "
prompt = torch.tensor([encode(prompt)], dtype = torch.long).to(device)
prompt.shape
generated_idx = model.generate(prompt, max_new_tokens=500)
output = decode(generated_idx[0].tolist())
output